## Dependencies

In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import numpy as np
from prepare_data import anc_path,pos_path,neg_path

### Setup and manage GPU usage

In [2]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    current_device= torch.cuda.current_device()
    # Limit memory usage to 80%
    torch.cuda.set_per_process_memory_fraction(0.8,device=current_device)
    # For optimized memory utilization during tensor operations
    torch.backends.cudnn.benchmark = True
    print(f"Using Device: {device}",
          f"\nCurrent GPU: {torch.cuda.get_device_name(current_device)}",
          f"\nCuda version: {torch.version.cuda}",
          f"\ncuDNN available: {torch.backends.cudnn.is_available()}",
          f"\ncuDNN version: {torch.backends.cudnn.version()}",
          f"\nAllocated memory: {torch.cuda.memory_allocated()} bytes",
          f"\nCached memory: {torch.cuda.memory_reserved()} bytes")
else:
    device = torch.device("cpu")
    print(f"Using Device: {device}")


Using Device: cuda 
Current GPU: NVIDIA GeForce RTX 4060 Laptop GPU 
Cuda version: 11.8 
cuDNN available: True 
cuDNN version: 90100 
Allocated memory: 0 bytes 
Cached memory: 0 bytes


## Register your face

In [ ]:
cap = 